# Convolutional Neural Networks

## Preliminaries

In this practical session you will implement and train several Convolutional Neural Networks (CNNs) using the Keras framework with a Tensorflow backend. If you are not already familiar with Keras, you can go over the [following tutorial](https://github.com/tueimage/essential-skills/blob/master/keras.md). More detailed information on the different functionalities can be found in the [Keras library documentation](https://keras.io/). 

Note that for this set of exercise CPU-only Tensorflow, which you should already have installed, is sufficient (i.e. GPU-support is not required but it will make your experiments run faster). 

You are also required to use the `gryds` package for data augmentation that you can install directly from git: `pip install git+https://github.com/tueimage/gryds/`.

You also have to install the Keras deep learning framework (if you have not done so already) by running `conda install keras`. Note that there are two implementations of Keras, one from https://keras.io/ and another one that ships with Tensorflow. Here we use the former. 

## MNIST
We will first train a simple CNN to classify handwritten digits using the MNIST dataset. This dataset is often referred to as the "Hello world!" example of deep learning because it can be used to quickly illustrate a small neural network in action (and obtain a decent classification accuracy in the process). More information on it can be found [here](http://yann.lecun.com/exdb/mnist/).

First, let's load the dataset and visualize some images.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

# load the MNIST the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# scale the image intensities to the 0-1 range
x_train = (x_train / 255.0).astype(np.float32)
x_test = (x_test / 255.0).astype(np.float32)

# convert the data to channel-last
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# convert the labels to one-hot encoded
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

def plot_images(images, dim=(10, 10), figsize=(10, 10), title=''):
    
    plt.figure(figsize=figsize)
    
    for i in range(images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    
    plt.tight_layout()
    plt.suptitle(title)
    plt.show()
    
plot_images(x_train[np.random.randint(0, x_train.shape[0], size=100)].reshape(100, 28, 28))

The MNIST classification task is quite simple: given an image, predict the digit that it contains. Thus, this is a 10-class classification problem.

Let's define a simple network for the handwritten digit classification.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Compile and train the network (note that this could take a while).

In [ ]:
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Now that the model is trained, we can evaluate its performance on the independent test dataset.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
model2.add(Conv2D(128, (12,12), activation='relu'))
model2.add(Dropout(0.5))
model2.add(Conv2D(10, (1,1), activation='softmax'))
model2.add(Flatten())

model2.summary()

In [ ]:
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

model2.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

model2.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))

score2 = model2.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

## U-Net
The U-Net convolutional neural network architecture was first developed for biomedical image segmentation and is to this day one of the most widely used methods for image segmentation. The details of the architecture can be found in the [original paper](https://arxiv.org/abs/1505.04597). In this practical we will build and train a U-Net network that is able to segment blood vessels in retinal images. 

### Loading and visualizing the data
The data for this task is taken from the [DRIVE](https://www.isi.uu.nl/Research/Databases/DRIVE/index.php) database. It consists of photographs of the retina, where the goal is to segment the blood vessels within. The dataset has a total of 40 photographs, divided in 20 images for training and 20 for testing. The images corresponding to the DRIVE test set can be found [here](https://www.dropbox.com/s/zk51wgupimw7jd9/DRIVE.zip?dl=0).

Let's load the training set and visualize an image with the corresponding blood vessel segmentation. For training we will divide the data in a training and a validation set to monitor the model during training.

In [ ]:
%matplotlib inline
import sys
sys.path.append('code/')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from glob import glob
from unet_utils import load_data

# location of the DRIVE dataset
data_folder = 'C:/Users/stijn/Desktop/school/TUe/2022-2023/8dm50 Machine learning in medical imaging and biology/8dm50-machine-learning/practicals/data/DRIVE/'
train_paths = glob(data_folder + 'training/images/*.tif')
images, masks, segmentations = load_data(train_paths)

# print the shape of image dataset
print(images.shape)

plt.figure(figsize=(10, 8))

plt.subplot(1, 2, 1)
plt.title("Image of the retina")
plt.axis('off')
plt.imshow(images[0])

plt.subplot(1, 2, 2)
plt.title("Ground truth vessel segmentation")
plt.axis('off')
plt.imshow(segmentations[0][:, :, 0])

plt.show()

# divide in training and validation
train_images, val_images, train_masks, val_masks, train_segmentations, val_segmentations = train_test_split(
    images, masks, segmentations, test_size=0.2, random_state=7)

# print the shape of the training and valudation datasets
print(train_images.shape)
print(train_masks.shape)
print(train_segmentations.shape)
print(val_images.shape)
print(val_masks.shape)
print(val_segmentations.shape)

### Training a U-Net model

You are already provided with implementation of the U-Net architecture in `unet_model.py`. This is a modular implementation and can be used to generate U-Net architectures with a variety of hyperparameters such as depth and number of feature maps. Before using the model, examine the code and documentation and make sure that you understand all the details.

We will train a U-Net model using smaller patches extracted from the training images. Training the images on smaller patches requires less computation power and results in a more varied training dataset (it has the effect of data augmentation by image translation). Because a U-Net is a fully convolutional network it can be evaluated on inputs of different size (the output size will change according to the input size). Thus, although the model will be trained on smaller patches it can still be used to segment larger images with one pass through the model. 


In [ ]:
from unet_utils import extract_patches, preprocessing

# work with 32x32 patches
patch_size = (32, 32)

# 200 patches per image
patches_per_im = 200

# visualize a couple of patches as a visual check
patches, patches_segmentations = extract_patches(train_images, train_segmentations, patch_size, patches_per_im=1, seed=7)

print(patches.shape)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 12))

for i in range(0, 3):
    axes[i, 0].axis('off')
    axes[i, 0].imshow(patches[i])
    axes[i, 1].axis('off')
    axes[i, 1].imshow(patches_segmentations[i][:, :, 0])
    axes[i, 2].axis('off')
    axes[i, 2].imshow(patches[i+5])
    axes[i, 3].axis('off')
    axes[i, 3].imshow(patches_segmentations[i+5][:, :, 0])

plt.show()

In [ ]:
# pad the validation data to fit the U-Net model
# images of shape (584, 565) shape result in concatenation error due to the odd number of columns

print("Old shape:", val_images.shape)

val_images, val_masks, val_segmentations = preprocessing(
    val_images, 
    val_masks, 
    val_segmentations, 
    desired_shape=(584, 584))
    
print("New shape:", val_images.shape)

In [ ]:
import os
from unet_model import unet
from unet_utils import datagenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# use a single training image, to better demonstrate the effects of data augmentation
X_train, y_train = np.expand_dims(train_images[0], axis=0), np.expand_dims(train_segmentations[0], axis=0)

print(X_train.shape)
print(y_train.shape)

# hyperparameters
depth = 3
channels = 32
use_batchnorm = True
batch_size = 64
epochs = 250
steps_per_epoch = int(np.ceil((patches_per_im * len(train_images)) / batch_size))

# work with 32x32 patches
patch_size = (32, 32)
# 200 patches per image
patches_per_im = 200

# initialize model
model = unet(input_shape=(None, None, 3), depth=depth, channels=channels, batchnorm=use_batchnorm)

# print a summary of the model
# model.summary(line_length=120)

# compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# stop the training if the validation loss does not increase for 15 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# train the model with the data generator, and save the training history
history = model.fit_generator(datagenerator(X_train, y_train, patch_size, patches_per_im, batch_size),
                              validation_data=(val_images, val_segmentations),
                              steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1,
                              callbacks=[early_stopping])

### Visualizing the segmentation results

In [ ]:
# run the model on one test image and show the results
from unet_utils import preprocessing

# test data paths
impaths_test = glob(data_folder + 'test/images/*.tif')

# load data
test_images, test_masks, test_segmentations = load_data(impaths_test, test=True)

# pad the data to fit the U-Net model
test_images, test_masks, test_segmentations = preprocessing(test_images, test_masks, test_segmentations, 
                                                            desired_shape=(584, 584))

# use a single image to evaluate
X_test, y_test = np.expand_dims(test_images[0], axis=0), np.expand_dims(test_masks[0], axis=0)

# predict test samples
test_prediction = model.predict(X_test, batch_size=4)

# visualize the test result
plt.figure(figsize=(12, 10))

plt.subplot(1, 3, 1)
plt.title("Image of the retina")
plt.axis('off')
plt.imshow(test_images[0])

plt.subplot(1, 3, 2)
plt.title("Ground truth vessel segmentation")
plt.axis('off')
plt.imshow(test_segmentations[0][:, :, 0])

plt.subplot(1, 3, 3)
plt.title("Predicted vessel segmentation")
plt.axis('off')
plt.imshow(test_prediction[0, :, :, 0])

plt.show()

# Exercises

## Number of parameters

The first convolutional layer in the MNIST example has 320 parameters. The first fully connected layer has 1179,776 parameters. What do these parameters correspond to? 

<font color='#770a0a'>What is the general expression for the number of parameters of 1) a convolutional layer and 2) a fully-connected layer?</font>

**Answer on the first question:**

The 320 parameters in the first convolutional layer are $3*3=9$ weights per kernel plus a bias, meaning $10$ weights per kernel in total. With 32 kernels, this gives a total of 320 learnable weights.

As for the first fully connected layer: the last conv layer had an output of $36,864$ values in a $24*24*64$ shape. Max pooling with a $2*2$ kernel and stride of $2$ then reduces this to $9,216$ in a $12*12*64$ shape. The flatten layer then changes the shape of this to $9,216*1*1$. In the FC layer connected to this each of the $128$ neurons then has a connection to each of these inputs plus a bias value. This gives a total number of learnable weights of $128*9,216 + 128 = 1,179,776$.

### General expressions
**The Number of Parameters of a Conv Layer:**

*$W_c$ = Number of weights of the Conv Layer.*

*$B_c$ = Number of biases of the Conv Layer.*

*$P_c$ = Number of parameters of the Conv Layer.*

*$K$ = Size (width) of kernels used in the Conv Layer.*

*$N$ = Number of kernels.*

*$C$ = Number of channels of the input image.*

\begin{align}
        W_c &= K^2 \times C \times N \\
        B_c &= N \\
        P_c &= W_c + B_c
\end{align}

**Number of Parameters of a Fully Connected (FC) Layer connected to a Conv Layer:**

*$W_{cf}$ = Number of weights of a FC Layer which is connected to a Conv Layer.*

*$B_{cf}$ = Number of biases of a FC Layer which is connected to a Conv Layer.*

*$P_{cf}$ = = Number of parameters of a FC Layer which is connected to a Conv Layer.*

*$O$ = Size (width) of the output image of the **previous** Conv Layer.*

*$N$ = Number of kernels in the previous Conv Layer.*

*$F$ = Number of neurons in the FC Layer.*

\begin{align}
        W_{cf} &= O^2 \times N \times F \\
        B_{cf} &= F \\
        P_{cf} &= W_{cf} + B_{cf}
\end{align}

**The number of Parameters of a Fully Connected (FC) Layer connected to a FC Layer:**

*$W_{ff}$ = Number of weights of a FC Layer which is connected to an FC Layer.*

*$B_{ff}$ = Number of biases of a FC Layer which is connected to an FC Layer.*

*$P_{ff}$ = Number of parameters of a FC Layer which is connected to an FC Layer.*

*$F$ = Number of neurons in the FC Layer.*

*$F_{-1}$ = Number of neurons in the previous FC Layer.*

\begin{align}
        W_{ff} &= F_{-1} \times F \\
        B_{ff} &= F \\
        P_{ff} &= W_{ff} + B_{ff}
\end{align}

## Fully-convolutional MNIST model

Modify the model in the MNIST example in such a way that it only contains convolutional layers while keeping the same number of parameters. If you do the modification correctly, the two models will have the same behaviour (i.e. they will represent the same model, only with different implementation). Show this experimentally. 


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
model2.add(Conv2D(128, (12,12), activation='relu'))
model2.add(Dropout(0.5))
model2.add(Conv2D(10, (1,1), activation='softmax'))
model2.add(Flatten())

model2.summary()

In [ ]:
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

model2.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

model2.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))

score2 = model2.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

**Answer on the question:**

The amount of parameters are exactly the same in every layer and the amount of total amount of trainable parameters is also the same. The training time and final accuracy and loss are also very close to eachother, which can be explained by a slight difference in initialized values while using the same model. 


## U-Net architecture

<font color='#770a0a'> What is the role of the skip connections in the U-Net neural network architecture? Will it be possible to train the exact same architecture with the skip connections omitted? If yes, what would be the expected result? If no, what would be the cause of the error?</font>

**Answer on the question:**

Yes this would be possible, however this would require the size of the model to increase, without any extra information being added. This would lead to a larger model, but a significant decrease in performance over the model with the skip connections.

## Data augmentation

<font color='#770a0a'>Why does data augmentation result in less overfitting? Can data augmentation be applied to the test samples? If yes, towards what goal? If no, what is preventing that?</font>


***Answer***

With data augmentation the data size is increased. The model is less likely to overfit as it is unable to overfit on all the samples. Introducing slight variability in the data will therefore improve the generalizability of the model.  

Data augmentation can be used on the training set to increase the number of samples and to increase the variability of our training data on wicht to fit the model on. The validation data can be augmented as well as it indicates when the model is most generalizable. Generalization happens only when in training the model and not during testing or validation and therefore data augmentation is not necessarily needed.

The test set is used to score the model's real-world performance. Data augmentation in this case is not necessary as the model is trained on the augmented dataset which should represent variance in input that is encountered in the real-world samples. Therefore, data augmentation is not necessary on the test set.



*Part 2:*

Implement random brightness augmentation of the image data by adding a random offset to the image intensity before passing them trough the network at training time. Train a model with random brightness augmentation and compare it to the baseline above. 




In [ ]:
from sklearn.feature_extraction.image import extract_patches_2d
from unet_utils import adjustbrightnessImages
# Brightness augmentation is implemented in the function extract_patches_randBA
# Patches are created for which their brightness is then augmented by a value delta, which is randomly chosen from the specified
# range. Below an example is visualized.

# work with 32x32 patches
patch_size = (32, 32)

# 200 patches per image
patches_per_im = 200

# range in which delta is chosen for brightness adjustment, which is added to all components of the image
RBA_range = [-0.4,0.4]

examp_patches = extract_patches_2d(train_images[0], patch_size, max_patches=patches_per_im, random_state=1)

examp_patches_adj = adjustbrightnessImages(examp_patches,RBA_range)

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))

for i in range(0, 3):
    axes[i, 0].axis('off')
    axes[i, 0].imshow(examp_patches[i])
    axes[i, 1].axis('off')
    axes[i, 1].imshow(examp_patches_adj[i])

axes[0, 0].set_title('Original patch')
axes[0, 1].set_title('Adjusted patch')

plt.show()


In [ ]:
from unet_utils import datagenerator_randBA, datagenerator_randBA_spline

train_images, val_images, train_masks, val_masks, train_segmentations, val_segmentations = train_test_split(
    images, masks, segmentations, test_size=0.2, random_state=7)


val_images, val_masks, val_segmentations = preprocessing(
    val_images, 
    val_masks, 
    val_segmentations, 
    desired_shape=(584, 584))

# hyperparameters
depth = 3
channels = 32
use_batchnorm = True
batch_size = 64
epochs = 250

# work with 32x32 patches
patch_size = (32, 32)

# 200 patches per image
patches_per_im = 200

# range for random brightness augmentation
brange = [-0.5,0.5]

steps_per_epoch = int(np.ceil((patches_per_im * len(train_images)) / batch_size))


# initialize model
model = unet(input_shape=(None, None, 3), depth=depth, channels=channels, batchnorm=use_batchnorm)


In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# stop the training if the validation loss does not increase for 15 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


history = model.fit_generator(datagenerator_randBA(train_images, train_segmentations, patch_size, patches_per_im, brange, batch_size),
                              validation_data=(val_images, val_segmentations),
                              steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1,
                              callbacks=[early_stopping])


In [ ]:

score = model.evaluate(test_images, test_segmentations, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
# use a single image to evaluate
X_test, y_test = np.expand_dims(test_images[0], axis=0), np.expand_dims(test_masks[0], axis=0)

# predict test samples
test_prediction = model.predict(X_test, batch_size=4)

# visualize the test result
plt.figure(figsize=(12, 10))

plt.subplot(1, 3, 1)
plt.title("Image of the retina")
plt.axis('off')
plt.imshow(test_images[0])

plt.subplot(1, 3, 2)
plt.title("Ground truth vessel segmentation")
plt.axis('off')
plt.imshow(test_segmentations[0][:, :, 0])

plt.subplot(1, 3, 3)
plt.title("Predicted vessel segmentation")
plt.axis('off')
plt.imshow(test_prediction[0, :, :, 0])

plt.show()

*Part 3:*

Implement data augmentation procedure that in addition to brightness augmentation also performs b-spline geometric augmentation using the [`gryds`](https://github.com/tueimage/gryds) package (you can look at the documentation of the package for an example on how to do that). Compare the new model with the baseline and the model that only performs brightness augmentation.


In [ ]:
from sklearn.feature_extraction.image import extract_patches_2d
from unet_utils import bsplinetransform

# B-spline geometric augmentation is implemented in the function extract_patches_randBA_spline
# Patches are created for which their brightness is then augmented by a value delta, which is randomly chosen from the specified
# range. The image is transformed with the function bsplinetransform. Below an example is visualized.

# work with 32x32 patches
patch_size = (32, 32)

# 200 patches per image
patches_per_im = 200

# range in which delta is chosen for brightness adjustment, which is added to all components of the image
RBA_range = [-0.4,0.4]
bsplinerange = [-0.05,0.05]

examp_patches_im = extract_patches_2d(train_images[0], patch_size, max_patches=patches_per_im, random_state=1)
examp_patches_seg = np.expand_dims(extract_patches_2d(train_segmentations[0], patch_size, max_patches=patches_per_im, random_state=1),
                                   axis=-1)

# bspline does
examp_im_trans, examp_seg_trans = bsplinetransform(examp_patches_im, examp_patches_seg, bounds=bsplinerange)

examp_im_adj = adjustbrightnessImages(examp_im_trans,RBA_range)


fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(17, 17))

for i in range(0, 3):
    axes[i, 0].axis('off')
    axes[i, 0].imshow(examp_patches_im[i])
    axes[i, 1].axis('off')
    axes[i, 1].imshow(examp_im_trans[i])
    axes[i, 2].axis('off')
    axes[i, 2].imshow(examp_im_adj[i])
    axes[i, 3].axis('off')
    axes[i, 3].imshow(examp_patches_seg[i])
    axes[i, 4].axis('off')
    axes[i, 4].imshow(examp_seg_trans[i])
    
axes[0, 0].set_title('Original patch')
axes[0, 1].set_title('Adjusted patch: b-spline')
axes[0, 2].set_title('Adjusted patch: brightness')
axes[0, 3].set_title('Original segmentation')
axes[0, 4].set_title('Adjusted segmentation: b-spline')

plt.show()

In [ ]:

splinebounds = [-0.05,0.05]

model_3 = unet(input_shape=(None, None, 3), depth=depth, channels=channels, batchnorm=use_batchnorm)

model_3.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [ ]:

# stop the training if the validation loss does not increase for 15 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# train the model with the data generator, and save the training history
history_RBA_spline = model_3.fit_generator(datagenerator_randBA_spline(train_images, train_segmentations, patch_size, patches_per_im, brange, splinebounds, batch_size),
                              validation_data=(val_images, val_segmentations),
                              steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1,
                              callbacks=[early_stopping])

In [ ]:
score = model_3.evaluate(test_images, test_segmentations, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])